In [1]:
import pandas as pd 
import numpy as np
from pandas import pivot_table
import nltk
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer,sent_tokenize,word_tokenize
from nltk import RegexpParser
import re

train = pd.read_csv("data/FakeNews_Training_Set.csv", header=0, delimiter=",")
test = pd.read_csv("data/FakeNews_Test_Set.csv", header=0, delimiter=",")
label_mapping = {
           'REAL': 1,
           'FAKE': 0}

train['label'] = train['label'].map(label_mapping)
test['label'] = test['label'].map(label_mapping)

In [2]:
regexAllComboTags=['NP: {<CD><NN>}',
'NP: {<CD><CD>}',
'NP: {<CD><NN>}',
'NP: {<CD><NNPS>}',
'NP: {<CD><NNS>}',
'NP: {<CD><NNS>}',
'NP: {<DT><CD>}',
'NP: {<DT><JJ>}',
'NP: {<DT><JJS>}',
'NP: {<DT><NN>}',
'NP: {<DT><NNP>}',
'NP: {<DT><NNPS>}',
'NP: {<DT><NNS>}',
'NP: {<DT><RB>}',
'NP: {<DT><VBG>}',
'NP: {<DT><VBN>}',
'NP: {<FW><FW>}',
'NP: {<FW><NNP>}',
'NP: {<IN><CD>}',
'NP: {<IN><NN>}',
'NP: {<JJ><NN>}',
'NP: {<JJ><NNP>}',
'NP: {<JJ><NNPS>}',
'NP: {<JJ><NNS>}',
'NP: {<JJR><JJ>}',
'NP: {<JJR><NNS>}',
'NP: {<JJS><NNS>}',
'NP: {<NN>}',
'NP: {<NN><JJ>}',
'NP: {<NN><NN>}',
'NP: {<NN><NNP>}',
'NP: {<NN><NNS>}',
'NP: {<NN><POS>}',
'NP: {<NN><RB>}',
'NP: {<NNP>}',
'NP: {<NNP><CD>}',
'NP: {<NNP><NN>}',
'NP: {<NNP><NNP>}',
'NP: {<NNP><NNPS>}',
'NP: {<NNP><NNS>}',
'NP: {<NNP><POS>}',
'NP: {<NNPS><POS>}',
'NP: {<NNS>}',
'NP: {<NNS><NNS>}',
'NP: {<NNS><POS>}',
'NP: {<PDT><DT>}',
'NP: {<PRP><NN>}',
'NP: {<RB>}',
'NP: {<RB><CD>}',
'NP: {<RB><DT>}',
'NP: {<RB><JJ>}',
'NP: {<RB><NN>}',
'NP: {<RB><NNS>}',
'NP: {<UH>}',
'NP: {<VBG><NN>}',
'NP: {<VBG><NNS>}',
'NP: {<VBN><NN>}',
'NP: {<VBN><NNS>}',
'NP: {<WDT><NN>}',
'PP: {<IN><JJ>}',
'PP: {<IN><RB>}',
'RB: {<FW><FW>}',
'RB: {<IN>}',
'RB: {<IN><JJS>}',
'RB: {<JJ>}',
'RB: {<RB>}',
'RB: {<RB><IN>}',
'RB: {<RB><RB>}',
'RB: {<RB><RBR>}',
'VP: {<CD><NN>}',
'VP: {<CD><NNS>}',
'VP: {<JJ><JJ><JJ>}',
'VP: {<JJ><JJ><NN>}',
'VP: {<JJ><JJ><RB>}',
'VP: {<JJ><NN>}',
'VP: {<JJ><NN><JJ>}',
'VP: {<JJ><NN><VBN>}',
'VP: {<JJ><RB><JJ>}',
'VP: {<JJ><RB><VBN>}',
'VP: {<JJ><VBG><JJ>}',
'VP: {<VB><JJ>}',
'VP: {<VB><RB>}',
'VP: {<VBD><IN>}',
'VP: {<VBD><JJ>}',
'VP: {<VBD><RB>}',
'VP: {<VBG><JJ>}',
'VP: {<VBG><RB>}',
'VP: {<VBN><IN>}',
'VP: {<VBN><JJ>}',
'VP: {<VBP><JJ>}',
'VP: {<VBZ><IN>}',
'VP: {<VBZ><JJ>}',
'VP: {<VBZ><RB>}']

In [3]:
# Creates a list containing 5 lists, each of 8 items, all set to 0

def processGrammer(dfTrain):

    matrix = np.zeros(shape=(len(dfTrain.index),94))

    #print (matrix.shape)

    for index,row in dfTrain.iterrows():

        if index % 50 == 0:
            print (index)
        twlist=[]
        sentListTrain =  sent_tokenize(row['text'])
        matrix[index,0]=row['sno']

        for sent in sentListTrain:
            #print (sent)
            #recvListTrain = process_content(sent)
            words= word_tokenize(sent)
            recvListTrain=nltk.pos_tag(words)

            for tagindex,multiPOSCombo in enumerate(regexAllComboTags):
                cp = nltk.RegexpParser(multiPOSCombo)
                result = cp.parse(recvListTrain)
                
                for subtree in result:
                    if type(subtree) == nltk.tree.Tree:
                        matrix[index,tagindex + 1]=matrix[index,tagindex + 1] + 1
                        #print (tagindex + 1,index)


    matrix.astype(int)
    #np.savetxt("numpy.out", matrix, newline=" ")
    df = pd.DataFrame(matrix)
    df.rename(columns={0: 'sno'}, inplace=True)
    #print (df.columns)
    df_merged = df.merge(dfTrain[['sno','label']],how="inner",on='sno')
    
    print (df_merged.columns)
    
    return df_merged

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run


print ('Training data to Extract and Pivot')

postags_merged = processGrammer(train)
#postags_merged = postags_merged.drop(['sno', 'sno_1'], axis=1)

print ('Training model')

# print ( postags_merged )

forest = forest.fit( postags_merged.drop(['label','sno'], axis=1), postags_merged["label"] )

print ('Tesing data to Extract and Pivot')

test_postags_merged = processGrammer(test)
#test_postags_merged = test_postags_merged.drop(['sno', 'sno_1'], axis=1)

print ('Model Prediction started')

print ( test_postags_merged )

# Use the random forest to make sentiment label predictions
result = forest.predict(test_postags_merged.drop(['label', 'sno'], axis=1))

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(test_postags_merged['label'], result)
accuracy_score(test_postags_merged['label'], result)

In [ ]:
test_postags_merged.to_csv("test_postag1.csv", sep='\t')
postags_merged.to_csv("train_postag1.csv", sep='\t')

In [5]:

from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# instantiate a logistic regression model, and fit with X and y
lr = LogisticRegression()

print ('Training data to Extract and Pivot')
postags_merged = processGrammer(train)

lr = lr.fit(postags_merged.drop(['label','sno'], axis=1), postags_merged["label"])

print ('Tesing data to Extract and Pivot')
test_postags_merged = processGrammer(test)



Training data to Extract and Pivot
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
Index([  'sno',       1,       2,       3,       4,       5,       6,       7,
             8,       9,      10,      11,      12,      13,      14,      15,
            16,      17,      18,      19,      20,      21,      22,      23,
            24,      25,      26,      27,      28,      29,      30,      31,
            32,      33,      34,      35,      36,      37,      38,      39,
            40,      41,      42,      43,      44,      45,      46,      47,
            48

ValueError: labels ['label' 'sno'] not contained in axis

In [13]:
print (test_postags_merged.columns)
print (test_postags_merged.drop(['label', 'sno'],axis=1))
# check the accuracy on the training set
lr.score(postags_merged.drop(['label', 'sno'],axis=1), postags_merged['label'])
# predict class labels for the test set
predicted = lr.predict(test_postags_merged.drop(['label', 'sno'],axis=1))
# generate class probabilities
probs = lr.predict_proba(test_postags_merged.drop(['label', 'sno'],axis=1))


# generate evaluation metrics

print (metrics.confusion_matrix(test['label'], predicted))
print (metrics.classification_report(test['label'], predicted))
print (metrics.accuracy_score(test['label'], predicted))
print (metrics.roc_auc_score(test['label'], probs[:, 1]))

Index([  'sno',       1,       2,       3,       4,       5,       6,       7,
             8,       9,      10,      11,      12,      13,      14,      15,
            16,      17,      18,      19,      20,      21,      22,      23,
            24,      25,      26,      27,      28,      29,      30,      31,
            32,      33,      34,      35,      36,      37,      38,      39,
            40,      41,      42,      43,      44,      45,      46,      47,
            48,      49,      50,      51,      52,      53,      54,      55,
            56,      57,      58,      59,      60,      61,      62,      63,
            64,      65,      66,      67,      68,      69,      70,      71,
            72,      73,      74,      75,      76,      77,      78,      79,
            80,      81,      82,      83,      84,      85,      86,      87,
            88,      89,      90,      91,      92,      93, 'label'],
      dtype='object')
        1    2     3    4     5     6 

In [16]:
type (probs)

numpy.ndarray

In [25]:
dfProbs = pd.DataFrame(probs)
dfProbs['sno']=test['sno']
dfProbs['label']=test['label']
dfProbs['text']=test['text'].str.replace('\n', '')

In [26]:
dfProbs.to_csv("posttag_probs.csv")

In [27]:
sample_fake_story="“What difference, at this point, does it make?” Remember those infamous and wretched words? Of course you do. We all do. Those were the words uttered by Hillary Clinton when she was desperately trying to spin her way through a committee hearing on Benghazi.And as anyone not named Hillary Clinton or anyone not on her corrupt payroll knows, it makes a lot of difference . This because Americans died. They died at the hands of terrorists on Hillary’s watch.New information is now out about some of the individuals who were paid to guard/protect state department employees at the special missions compound where the terrorist attack occurred and it’s not pretty. Turns out Hillary’s state department spent $9.2 million on a contract for guards, many of whom turned out to be the very terrorists who attacked the compound, killing our ambassador and other Americans in the process.Via Fox News .“Many of the local Libyans who attacked the consulate on the night of Sept. 11, 2012, were the actual guards that the State Department under Hillary Clinton hired to protect the Consulate in Benghazi,” Tiegen told Fox News. “The guards were unvetted and were locals with basically no background at all in providing security. Most of them never had held a job in security in the past.“Blue Mountain Libya, at the time of being awarded the contract by our State Department, had no employees so they quickly had to find people to work, regardless of their backgrounds,” he said.One former guard who witnessed the attack, Weeam Mohamed, confirmed in an email sent to the Citizens Commission on Benghazi and obtained by Fox News, that at least four of the guards hired by Blue Mountain took part in the attack after opening doors to allow their confederates in.“In the U.S. Mission, there were four people [who] belonged to the battalion February 17,” Mohamed wrote to the Commission, an independent body formed with Accuracy in Media to investigate the attack and the administration’s handling of it.Again, it makes a lot of difference."

In [57]:
matrix = np.zeros(shape=(1,94))
sentListTrain =  sent_tokenize(sample_fake_story)
matrix[0,0]=123

for sent in sentListTrain:
    words= word_tokenize(sent)
    recvListTrain=nltk.pos_tag(words)

    for tagindex,multiPOSCombo in enumerate(regexAllComboTags):
        cp = nltk.RegexpParser(multiPOSCombo)
        result = cp.parse(recvListTrain)
                
        for subtree in result:
            if type(subtree) == nltk.tree.Tree:
                matrix[0,tagindex + 1]=matrix[0,tagindex + 1] + 1

matrix.astype(int)
#np.savetxt("numpy.out", matrix, newline=" ")
dfPredict = pd.DataFrame(matrix)
dfPredict.rename(columns={0: 'sno'}, inplace=True)

print (dfPredict)

     sno    1    2    3    4    5    6    7    8    9 ...    84   85   86  \
0  123.0  0.0  1.0  0.0  0.0  1.0  1.0  0.0  8.0  0.0 ...   2.0  1.0  0.0   

    87   88   89   90   91   92   93  
0  0.0  3.0  0.0  0.0  0.0  0.0  1.0  

[1 rows x 94 columns]


In [58]:
testDF = pd.DataFrame([])
print ("Before Changes: Prediction : ",lr.predict_proba(dfPredict.drop(['sno'],axis=1)))
for i in range(1,93):
    testDf = dfPredict
    testDf[i] = testDf[i] + 1
    print ("Feature No : ",i," : ",lr.predict_proba(testDf.drop(['sno'],axis=1)))


Before Changes: Prediction :  [[ 0.53570821  0.46429179]]
Feature No :  1  :  [[ 0.53030462  0.46969538]]
Feature No :  2  :  [[ 0.54176573  0.45823427]]
Feature No :  3  :  [[ 0.53637162  0.46362838]]
Feature No :  4  :  [[ 0.44468354  0.55531646]]
Feature No :  5  :  [[ 0.44696536  0.55303464]]
Feature No :  6  :  [[ 0.44924941  0.55075059]]
Feature No :  7  :  [[ 0.43617831  0.56382169]]
Feature No :  8  :  [[ 0.4409141  0.5590859]]
Feature No :  9  :  [[ 0.41570653  0.58429347]]
Feature No :  10  :  [[ 0.39509475  0.60490525]]
Feature No :  11  :  [[ 0.40516972  0.59483028]]
Feature No :  12  :  [[ 0.45579354  0.54420646]]
Feature No :  13  :  [[ 0.47896305  0.52103695]]
Feature No :  14  :  [[ 0.47311185  0.52688815]]
Feature No :  15  :  [[ 0.46181499  0.53818501]]
Feature No :  16  :  [[ 0.46443321  0.53556679]]
Feature No :  17  :  [[ 0.49704964  0.50295036]]
Feature No :  18  :  [[ 0.52837561  0.47162439]]
Feature No :  19  :  [[ 0.5169004  0.4830996]]
Feature No :  20  :  [[ 